# Demo Process Synchronization
Next to the synchronization of activities using controll structures like **while** and **sequence** used within a single process, OpenCLSim allows to synchronize different processes using **start_events**. Start_events can be specified using the expression language as documented to **delay** the execution of a process. Control structures do not delay the execution but explicitly start the execution when possible. The difference is very well visible when you compare the activity log in this demo with the one from the sequence demo.

In [1]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of two basic activities
The two activities are started in parallel, but the execution of Activity2 is delayed until Activity1 has been completed. To easier see how the two activities are interrelated a reporting activity is added.

In [2]:
reporting_activity_data = {
    "env": my_env,
    "name": "Reporting activity",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5k",  # For logging purposes
    "registry": registry,
    "duration": 0,
    "postpone_start": False,
}
reporting_activity = model.BasicActivity(**reporting_activity_data)

basic_activity_data= {"env"  : my_env,
                      "name" : "Activity1",
                      "ID":"6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
                      "registry": registry,
                      "additional_logs": [reporting_activity],
                      "duration" : 14}
activity = model.BasicActivity(**basic_activity_data)
basic_activity_data2= {"env"  : my_env,
                      "name" : "Activity2",
                      "ID":"6dbbbdf7-4589-11e9-bf3b-b469212bff5c",  # For logging purposes
                      "registry": registry,
                      "additional_logs": [reporting_activity],
                      "start_event": [{"type":"activity", "name": "Activity1" , "state":"done"}],
                      "duration" : 30}
activity2 = model.BasicActivity(**basic_activity_data2)


## Run simulation

In [3]:
my_env.run()

In [4]:
log_df = pd.DataFrame(reporting_activity.log)
data =log_df[['Message', 'ActivityState', 'Timestamp', 'Value', 'ActivityID']]
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,Reporting activity,START,1970-01-01 00:00:00,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5k
1,Activity1,START,1970-01-01 00:00:00,14,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
2,Activity2,WAIT_START,1970-01-01 00:00:00,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
3,Reporting activity,STOP,1970-01-01 00:00:00,0,6dbbbdf7-4589-11e9-bf3b-b469212bff5k
4,Activity1,STOP,1970-01-01 00:00:14,14,6dbbbdf7-4589-11e9-bf3b-b469212bff5b
5,Activity2,WAIT_STOP,1970-01-01 00:00:14,-1,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
6,Activity2,START,1970-01-01 00:00:14,30,6dbbbdf7-4589-11e9-bf3b-b469212bff5c
7,Activity2,STOP,1970-01-01 00:00:44,30,6dbbbdf7-4589-11e9-bf3b-b469212bff5c


Both activities start at the same time. Activity2 gets into a waiting state, which ends, when Activity1 ends. Then Activity2 is executed.